In [27]:
# learning curve, fitting,feature importance. rmse,mse, etc..
# 

import pandas as pd
# https://www.kaggle.com/code/rtatman/lightgbm-hyperparameter-optimisation-lb-0-761/notebook
import xgboost as xgb
import lightgbm as lgb
import numpy as np
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import warnings
warnings.filterwarnings('ignore')

In [29]:
df = pd.read_excel('budget_data.xlsx')

df = df[~df['CUST_SALARY_RANGE_TIER'].isin(['01. Unknown','6. 5000+'])]

In [33]:
df['CUST_SALARY_RANGE_TIER'].value_counts()

2. 500-1000         11012
3. 1000-1500        10841
1. Less than 500     8054
4. 1500-2000         5646
5. 2000-5000         4223
6. 5000+              194
01. Unknown            25
Name: CUST_SALARY_RANGE_TIER, dtype: int64

In [3]:
def f(x):
    if 'Less' in x:
        return (0 + int(x.split()[-1]))/2
    else:
        y = x.split()[-1]
        y = y.split('-')
        y = int(y[0]) + int(y[1])
        return y/2

In [4]:
df['Salary_temp'] = df['CUST_SALARY_RANGE_TIER'].apply(lambda x: f(x))

In [5]:
df

,date,Customer number,nationality,customer_category,NO_OF_ALL_CC,NO_OF_SAL_HITS,GENDER_DESC,SALARY,CUST_SALARY_RANGE_TIER,CUST_AGE,CUST_AGE_TIER,NO_OF_ALL_ACC,BAL_ALL_ACC,IS_CUST_HOLDING_PF,IS_CUST_HOLDING_CC,IS_CUST_HOLDING_PRE_PAID,IS_CUST_WITH_SAL_TRF,NO_OF_POS_TXS,ALL_POS_TXS,NO_OF_ALL_PRODUCTS,NO_OF_ALL_PRODUCT_CATEGORIES,CUST_SEGMENT_BAL,SAL_AMOUNT,Groceries,Transportation,Health,Utilities,DiningOut,BillsAndServices,Shopping,Entertainment,Salary_temp
0,2021-11-30,1000388,2,5,0,1,Male,1000.0,3. 1000-1500,39.0,5. 30-40,1,3771.373047,0,0,0,0,53,2493.289,1,1,2. Smart Saver,0.000,50.000,5.00,10.000,32.0,6.85,0.00,605.79,20.00,1250.0
1,2021-11-30,1000444,1,5,0,2,Female,1454.0,3. 1000-1500,39.4,5. 30-40,1,102.569000,0,0,0,0,8,280.555,1,1,1. Basic Saver,43.170,30.205,0.00,0.000,54.1,46.25,0.00,0.00,0.00,1250.0
2,2021-11-30,1000756,1,2,0,0,Male,1350.0,3. 1000-1500,37.3,5. 30-40,1,37805.996094,0,0,0,0,56,2742.224,1,1,2. Smart Saver,0.000,50.524,0.00,99.445,30.0,42.80,892.00,222.05,0.00,1250.0
3,2021-11-30,1000856,2,5,0,0,Male,600.0,2. 500-1000,64.8,8. >=60,2,438.446990,0,0,0,0,29,422.390,2,1,2. Smart Saver,0.000,101.250,100.00,21.770,0.0,6.50,118.16,7.26,0.00,750.0
4,2021-11-30,1000901,1,2,0,6,Male,1950.0,4. 1500-2000,56.7,7. 50-60,3,5171.171844,0,0,0,1,15,1850.505,3,1,2. Smart Saver,1658.301,40.015,54.25,120.000,0.0,6.20,36.00,482.19,49.35,1750.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39990,2022-03-31,1288484,2,4,0,0,Female,500.0,2. 500-1000,22.8,4. 21-30,1,8.005000,0,0,0,0,40,939.430,1,1,1. Inactive,0.000,1.500,0.00,0.000,6.2,115.13,5.00,48.85,14.25,750.0
39991,2022-03-31,1288496,2,5,0,1,Female,716.0,2. 500-1000,35.6,5. 30-40,1,3000.531982,0,0,0,0,0,0.000,1,1,2. Smart Saver,50.000,0.000,0.00,0.000,0.0,0.00,0.00,0.00,0.00,750.0
39992,2022-03-31,1288635,1,5,0,0,Male,1500.0,4. 1500-2000,57.1,7. 50-60,1,158.735992,0,0,0,0,51,321.045,1,1,2. Smart Saver,0.000,175.145,53.00,53.950,0.0,33.95,4.00,0.00,0.00,1750.0
39993,2022-03-31,1288723,2,5,0,2,Male,0.0,1. Less than 500,29.8,4. 21-30,1,33.990002,0,0,0,1,116,2739.215,1,1,2. Smart Saver,140.000,83.485,3.00,17.200,13.0,265.20,24.90,627.21,35.15,250.0


In [6]:
temp_salary = []
for _, row in df.iterrows():
    if row['SALARY']==0:
        x = (row['SAL_AMOUNT'] if row['SAL_AMOUNT']!=0 else row['Salary_temp'])
    else:
        x = row['SALARY']
    
    temp_salary.append(x)

In [7]:
df['SALARY'] = temp_salary

In [8]:
df.drop('SAL_AMOUNT',axis=1,inplace=True)
df.drop('Salary_temp',axis=1,inplace=True)
df.drop('CUST_SALARY_RANGE_TIER',axis=1,inplace=True)
df.drop('date',axis=1,inplace=True)
df.drop('Customer number',axis=1,inplace=True)

In [9]:
df.drop('CUST_AGE_TIER',axis=1,inplace=True)

In [10]:
df

,nationality,customer_category,NO_OF_ALL_CC,NO_OF_SAL_HITS,GENDER_DESC,SALARY,CUST_AGE,NO_OF_ALL_ACC,BAL_ALL_ACC,IS_CUST_HOLDING_PF,IS_CUST_HOLDING_CC,IS_CUST_HOLDING_PRE_PAID,IS_CUST_WITH_SAL_TRF,NO_OF_POS_TXS,ALL_POS_TXS,NO_OF_ALL_PRODUCTS,NO_OF_ALL_PRODUCT_CATEGORIES,CUST_SEGMENT_BAL,Groceries,Transportation,Health,Utilities,DiningOut,BillsAndServices,Shopping,Entertainment
0,2,5,0,1,Male,1000.0,39.0,1,3771.373047,0,0,0,0,53,2493.289,1,1,2. Smart Saver,50.000,5.00,10.000,32.0,6.85,0.00,605.79,20.00
1,1,5,0,2,Female,1454.0,39.4,1,102.569000,0,0,0,0,8,280.555,1,1,1. Basic Saver,30.205,0.00,0.000,54.1,46.25,0.00,0.00,0.00
2,1,2,0,0,Male,1350.0,37.3,1,37805.996094,0,0,0,0,56,2742.224,1,1,2. Smart Saver,50.524,0.00,99.445,30.0,42.80,892.00,222.05,0.00
3,2,5,0,0,Male,600.0,64.8,2,438.446990,0,0,0,0,29,422.390,2,1,2. Smart Saver,101.250,100.00,21.770,0.0,6.50,118.16,7.26,0.00
4,1,2,0,6,Male,1950.0,56.7,3,5171.171844,0,0,0,1,15,1850.505,3,1,2. Smart Saver,40.015,54.25,120.000,0.0,6.20,36.00,482.19,49.35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39990,2,4,0,0,Female,500.0,22.8,1,8.005000,0,0,0,0,40,939.430,1,1,1. Inactive,1.500,0.00,0.000,6.2,115.13,5.00,48.85,14.25
39991,2,5,0,1,Female,716.0,35.6,1,3000.531982,0,0,0,0,0,0.000,1,1,2. Smart Saver,0.000,0.00,0.000,0.0,0.00,0.00,0.00,0.00
39992,1,5,0,0,Male,1500.0,57.1,1,158.735992,0,0,0,0,51,321.045,1,1,2. Smart Saver,175.145,53.00,53.950,0.0,33.95,4.00,0.00,0.00
39993,2,5,0,2,Male,140.0,29.8,1,33.990002,0,0,0,1,116,2739.215,1,1,2. Smart Saver,83.485,3.00,17.200,13.0,265.20,24.90,627.21,35.15


In [11]:
df['GENDER_DESC'] = df['GENDER_DESC'].replace(['Male','Female'],[1,0])

In [12]:
df['CUST_SEGMENT_BAL'].value_counts().index

Index(['1. Basic Saver', '2. Smart Saver', '1. Inactive', '3. Long Term Planner'], dtype='object')

In [13]:
df = pd.get_dummies(df, columns=['CUST_SEGMENT_BAL'])

In [14]:
df

,nationality,customer_category,NO_OF_ALL_CC,NO_OF_SAL_HITS,GENDER_DESC,SALARY,CUST_AGE,NO_OF_ALL_ACC,BAL_ALL_ACC,IS_CUST_HOLDING_PF,IS_CUST_HOLDING_CC,IS_CUST_HOLDING_PRE_PAID,IS_CUST_WITH_SAL_TRF,NO_OF_POS_TXS,ALL_POS_TXS,NO_OF_ALL_PRODUCTS,NO_OF_ALL_PRODUCT_CATEGORIES,Groceries,Transportation,Health,Utilities,DiningOut,BillsAndServices,Shopping,Entertainment,CUST_SEGMENT_BAL_1. Basic Saver,CUST_SEGMENT_BAL_1. Inactive,CUST_SEGMENT_BAL_2. Smart Saver,CUST_SEGMENT_BAL_3. Long Term Planner
0,2,5,0,1,1,1000.0,39.0,1,3771.373047,0,0,0,0,53,2493.289,1,1,50.000,5.00,10.000,32.0,6.85,0.00,605.79,20.00,0,0,1,0
1,1,5,0,2,0,1454.0,39.4,1,102.569000,0,0,0,0,8,280.555,1,1,30.205,0.00,0.000,54.1,46.25,0.00,0.00,0.00,1,0,0,0
2,1,2,0,0,1,1350.0,37.3,1,37805.996094,0,0,0,0,56,2742.224,1,1,50.524,0.00,99.445,30.0,42.80,892.00,222.05,0.00,0,0,1,0
3,2,5,0,0,1,600.0,64.8,2,438.446990,0,0,0,0,29,422.390,2,1,101.250,100.00,21.770,0.0,6.50,118.16,7.26,0.00,0,0,1,0
4,1,2,0,6,1,1950.0,56.7,3,5171.171844,0,0,0,1,15,1850.505,3,1,40.015,54.25,120.000,0.0,6.20,36.00,482.19,49.35,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39990,2,4,0,0,0,500.0,22.8,1,8.005000,0,0,0,0,40,939.430,1,1,1.500,0.00,0.000,6.2,115.13,5.00,48.85,14.25,0,1,0,0
39991,2,5,0,1,0,716.0,35.6,1,3000.531982,0,0,0,0,0,0.000,1,1,0.000,0.00,0.000,0.0,0.00,0.00,0.00,0.00,0,0,1,0
39992,1,5,0,0,1,1500.0,57.1,1,158.735992,0,0,0,0,51,321.045,1,1,175.145,53.00,53.950,0.0,33.95,4.00,0.00,0.00,0,0,1,0
39993,2,5,0,2,1,140.0,29.8,1,33.990002,0,0,0,1,116,2739.215,1,1,83.485,3.00,17.200,13.0,265.20,24.90,627.21,35.15,0,0,1,0


In [15]:
col = ['Groceries','Transportation','Health','Utilities','DiningOut','BillsAndServices','Shopping','Entertainment']
X = df.drop(col,axis=1).values

from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X = sc_X.fit_transform(X)

In [16]:
# y = df[col].values

In [17]:
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

param_test ={'num_leaves': sp_randint(6, 50), 
             'min_child_samples': sp_randint(100, 500), 
             'min_child_weight': [1e-5, 1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3, 1e4],
             'subsample': sp_uniform(loc=0.2, scale=0.8), 
             'colsample_bytree': sp_uniform(loc=0.4, scale=0.6),
             'reg_alpha': [0, 1e-1, 1, 2, 5, 7, 10, 50, 100],
             'reg_lambda': [0, 1e-1, 1, 5, 10, 20, 50, 100]}

In [18]:
def model(X,y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

    clf = lgb.LGBMRegressor(objective="regression",metric="rmse",random_state=42, n_jobs=-1, n_estimators=500)
    gs = RandomizedSearchCV(
        estimator=clf, param_distributions=param_test, 
        n_iter=100,
        scoring='neg_mean_absolute_percentage_error',
        cv=3,
        refit=True,
        random_state=314,
        verbose=True)

    gs.fit(X_train, y_train)
    opt_parameters  = gs.best_params_

    print('opt_parameters:',opt_parameters)
    print('gs.best_score:',gs.best_score_)

    clf_sw = lgb.LGBMRegressor(**clf.get_params())
    #set optimal parameters
    clf_sw.set_params(**opt_parameters)
    clf_sw.fit(X_train, y_train)
    y_pred = clf_sw.predict(X_test)

    d = pd.DataFrame({'Groceries_TRUE':y_test,'Groceries_PRED':y_pred})
    return d

- 'Groceries','Transportation','Health','Utilities','DiningOut','BillsAndServices','Shopping','Entertainment'

# Groceries

In [19]:
d = model(X,df['Groceries'].values)

Fitting 3 folds for each of 100 candidates, totalling 300 fits
opt_parameters: {'colsample_bytree': 0.9501241488957805, 'min_child_samples': 301, 'min_child_weight': 0.1, 'num_leaves': 28, 'reg_alpha': 0, 'reg_lambda': 100, 'subsample': 0.9326466073236168}
gs.best_score: -2.4387892950252308e+16


In [20]:
d

,Groceries_TRUE,Groceries_PRED
0,22.500,9.842038
1,62.525,115.580314
2,35.840,110.120415
3,96.075,76.156278
4,2.600,44.251380
...,...,...
7951,0.000,20.303258
7952,0.000,23.921696
7953,50.040,23.603715
7954,0.000,0.210297


# Transportation

In [21]:
d = model(X,df['Transportation'].values)

Fitting 3 folds for each of 100 candidates, totalling 300 fits
opt_parameters: {'colsample_bytree': 0.9904687599207201, 'min_child_samples': 468, 'min_child_weight': 0.1, 'num_leaves': 42, 'reg_alpha': 0, 'reg_lambda': 100, 'subsample': 0.849377423799113}
gs.best_score: -1.0160484695608774e+17


In [22]:
d

,Groceries_TRUE,Groceries_PRED
0,0.000,9.968603
1,28.800,21.676965
2,84.349,51.501579
3,0.000,-2.431270
4,0.300,9.290943
...,...,...
7951,0.000,-3.310116
7952,0.000,44.987004
7953,0.000,16.198240
7954,0.000,4.500867


# Health

In [23]:
d = model(X,df['Health'].values)

Fitting 3 folds for each of 100 candidates, totalling 300 fits
opt_parameters: {'colsample_bytree': 0.8859275961007896, 'min_child_samples': 373, 'min_child_weight': 10000.0, 'num_leaves': 40, 'reg_alpha': 7, 'reg_lambda': 10, 'subsample': 0.7130182108881609}
gs.best_score: -4.154160065071658e+16


In [24]:
d

,Groceries_TRUE,Groceries_PRED
0,12.150,4.525475
1,12.750,4.510147
2,268.194,113.611035
3,218.500,51.852809
4,8.000,45.518922
...,...,...
7951,0.000,14.840996
7952,13.485,15.233381
7953,4.000,15.504095
7954,0.000,7.608914


# Utilities

In [25]:
d = model(X,df['Utilities'].values)

Fitting 3 folds for each of 100 candidates, totalling 300 fits
opt_parameters: {'colsample_bytree': 0.8900312305815961, 'min_child_samples': 258, 'min_child_weight': 1000.0, 'num_leaves': 22, 'reg_alpha': 10, 'reg_lambda': 20, 'subsample': 0.29153347452280826}
gs.best_score: -4.341583222031051e+16


In [26]:
d

,Groceries_TRUE,Groceries_PRED
0,8.000,13.277769
1,51.340,28.082050
2,114.148,91.329680
3,25.000,37.010336
4,0.000,10.815166
...,...,...
7951,46.000,28.216146
7952,0.000,6.583364
7953,0.000,36.629004
7954,0.000,7.581929
